In [19]:
from randdiag import *
from time import time

In [21]:
def offdiagonal_frobenius(A):
    loss = np.linalg.norm(A - np.diag(np.diagonal(A)),'fro')
    return loss

def compute_distance_eigenvalue_unitary(D1,D2):
    s = 0
    for d in D1:
        s+= np.min(np.absolute(D2-d))
    return s / D1.size

def compare_algorithms(ns = [10,100,1000],repeats = 100):
    for n in ns:
        print('Matrix size: ', n)
        A = np.random.randn(n,n) + 1j*np.random.randn(n,n)
        U,_ = np.linalg.qr(A)
        rt_rjd = 0
        err_rjd = 0
        for _ in range(repeats):
            start = time()
            Q = randdiag(U)
            rt_rjd += time()-start
            err_rjd += offdiagonal_frobenius(Q.conj().T @ U @ Q)
        print("RandDiag: {:.2f}, {:.2e}".format( rt_rjd / repeats, err_rjd / repeats))
        rt_schur = 0
        err_schur = 0
        for _ in range(repeats):
            start = time()
            T,Z = schur(U, 'complex')
            rt_schur+=time()-start
            err_schur += offdiagonal_frobenius(Z.conj().T @ U @ Z)
            #print(np.linalg.norm(Z.conj().T @ Z -np.eye(n)))
        print("SCHUR: {:.2f}, {:.2e}".format( rt_schur / repeats, err_schur / repeats))
        rt_schur = 0
        for _ in range(repeats):
            start = time()
            D = eigvals(U)
            rt_schur+=time()-start
        print("Schur Eigenvalue only: {:.2f}".format( rt_schur / repeats))
        rt_hess = 0
        for _ in range(repeats):
            start = time()
            D = eigenvalue_unitary_angle(U)
            rt_hess+=time()-start
        print("Rand Eigval: {:.2f}".format( rt_hess / repeats))

In [3]:
compare_algorithms(ns = [100],repeats=1)

Matrix size:  100
RandDiag: 0.01, 1.25e-12
SCHUR: 0.02, 4.89e-14
Schur Eigenvalue only: 0.01
Rand Eigval: 0.00


In [4]:
def compare_algorithms_eigenvalue(ns = [10,100,1000],repeats = 100):
    for n in ns:
        print('Matrix size: ', n)
        A = np.random.randn(n,n) + 1j*np.random.randn(n,n)
        U,_ = np.linalg.qr(A)
        rt_schur = 0
        err_schur = 0
        D_true = eigvals(U)
        rt_rjd = 0
        err_rjd = 0
        for _ in range(repeats):
            start = time()
            Q = randdiag(U)
            D = np.diag(Q.conj().T @ U @ Q)
            rt_rjd += time()-start
            err_rjd += compute_distance_eigenvalue_unitary(D,D_true)
        print("RandDiag: {:.2f}, {:.2e}".format( rt_rjd / repeats, err_rjd / repeats))
        for _ in range(repeats):
            start = time()
            D = eigvals(U)
            rt_schur+=time()-start
            err_schur += compute_distance_eigenvalue_unitary(D ,D_true)
        print("SCHUR: {:.2f}, {:.2e}".format( rt_schur / repeats, err_schur / repeats))
        rt_eigval = 0
        err_eigval = 0
        for _ in range(repeats):
            start = time()
            D = eigenvalue_unitary_angle(U)
            rt_eigval+=time()-start
            err_eigval+= compute_distance_eigenvalue_unitary(np.exp(1j*D),D_true)
        print("Rand Eigval: {:.2f}, {:.2e}".format( rt_eigval / repeats,err_eigval / repeats))
compare_algorithms_eigenvalue([1000],100)

Matrix size:  1000
RandDiag: 0.47, 4.36e-15
SCHUR: 1.42, 0.00e+00
Rand Eigval: 0.55, 6.16e-15


In [5]:
A = np.random.randn(1000,1000) + 1j*np.random.randn(1000,1000)
U,_ = np.linalg.qr(A)
rt_hess = 0
start = time()
Q,H = hessenberg(U,calc_q=True)
rt_hess = time() - start

rt_randdiag = 0
start = time()
randdiag(U)
rt_randdiag = time() - start

print("Hess rt: ", rt_hess, "RandDiag rt: ", rt_randdiag)

Hess rt:  0.32164502143859863 RandDiag rt:  0.41239500045776367


In [ ]:
from randdiag import *
def eigenvalue_error_experiment(n=1000,repeats = 100, schur_transformed = True):
    N, d_reference = random_normal_matrix(n)
    errors_randdiag = np.zeros(repeats)
    errors_schur = np.zeros(repeats)
    for i in range(repeats):
        U = randdiag(N)
        d_computed_randdiag = np.diag(U.conj().T @ N @ U)
        _, error = compare_eigenvalues(d_reference, d_computed_randdiag)
        errors_randdiag[i] = error

        T,Z = schur(N, 'complex')
        if schur_transformed:
            d_computed_schur = np.diag(Z.conj().T @ N @ Z)
        else:
            d_computed_schur = np.diag(T)
        _, error = compare_eigenvalues(d_reference, d_computed_schur)
        errors_schur[i] = error
    mean, std, min_error, max_error = report_stats(errors_randdiag)
    print(f'Matrix size: {n}')
    print(f'Randdiag Mean: {mean}, Std: {std}, Min: {min_error}, Max: {max_error}')
    mean, std, min_error, max_error = report_stats(errors_schur)
    print(f'Schur Mean: {mean}, Std: {std}, Min: {min_error}, Max: {max_error}')




IndentationError: expected an indented block after 'if' statement on line 12 (1617839694.py, line 13)

In [15]:
eigenvalue_error_experiment(500,100)
eigenvalue_error_experiment(1000,100)
eigenvalue_error_experiment(1500,100)

Matrix size: 500
Randdiag Mean: 1.1223722589431645e-15, Std: 3.295510250434284e-17, Min: 1.0550885455270185e-15, Max: 1.2206966374553545e-15
Schur Mean: 4.7414317050894804e-15, Std: 7.888609052210118e-31, Min: 4.741431705089481e-15, Max: 4.741431705089481e-15
Matrix size: 1000
Randdiag Mean: 1.5670242578363867e-15, Std: 3.9639087004544605e-17, Min: 1.4656844216222907e-15, Max: 1.6706534966850734e-15
Schur Mean: 5.911517465276458e-15, Std: 3.1554436208840472e-30, Min: 5.911517465276461e-15, Max: 5.911517465276461e-15
Matrix size: 1500
Randdiag Mean: 1.4944710408166517e-15, Std: 2.5640389902190834e-17, Min: 1.4165227270679788e-15, Max: 1.5525480837830273e-15
Schur Mean: 6.6067287135361205e-15, Std: 1.5777218104420236e-30, Min: 6.606728713536122e-15, Max: 6.606728713536122e-15


In [23]:
def compute_error(n, repeats):
    A = np.random.randn(n,n) + 1j*np.random.randn(n,n)
    U,_ = np.linalg.qr(A)
    err_rjd = 0
    for i in range(repeats):
        Q = randdiag(U)
        err_rjd += offdiagonal_frobenius(Q.conj().T @ U @ Q)
    return err_rjd / repeats

In [ ]:
sizes = range(1,13)
sizes = [ 2**x for x in sizes]
errors = []
repeats = 100
for size in sizes:
    errors.append(compute_error(size,repeats))
errors = np.array(errors)

np.save('error_vs_size.npy', errors)


KeyboardInterrupt: 

In [ ]:
print(errors)
print(sizes)
errors[2] = compute_error(8,repeats)
print(errors)


[np.float64(3.9623919712379776e-16), np.float64(1.5932589059875236e-14), np.float64(7.879015535785928e-11), np.float64(3.319032922926852e-13), np.float64(4.390998332731221e-12), np.float64(6.627445014958745e-12), np.float64(3.4669218171467926e-11), np.float64(2.8754214380100394e-09)]
[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]
[np.float64(3.9623919712379776e-16), np.float64(1.5932589059875236e-14), np.float64(9.057463843287082e-14), np.float64(3.319032922926852e-13), np.float64(4.390998332731221e-12), np.float64(6.627445014958745e-12), np.float64(3.4669218171467926e-11), np.float64(2.8754214380100394e-09)]


In [29]:
np.save('error_vs_size_new.npy', errors)

# Code below is draft

In [7]:
print(col_ind,row_ind)

[1 0 2] [0 1 2]


In [15]:
n = 1000
A = np.random.randn(n,n)
U,_ = np.linalg.qr(A)
repeats = 1

In [137]:
rt_rjd = 0
err_rjd = 0
for _ in range(repeats):
    start = time()
    H = (U+U.conj().T) / 2; S = (U-U.conj().T) / 2
    AA = np.array([H,1j*S])
    mu = np.random.normal(0,1,2)
    A_mu = np.einsum('ijk,i->jk',AA, mu)
    _, Q = np.linalg.eigh(A_mu)
    rt_rjd += time()-start
    err_rjd += offdiagonal_frobenius(Q.conj().T @ U @ Q)
print(rt_rjd / repeats, err_rjd / repeats)

0.6310639381408691 1.6923075836340513e-11


[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]


In [138]:
rt_schur = 0
err_schur = 0
for _ in range(repeats):
    start = time()
    T,Z = schur(U, 'complex')
    rt_schur+=time()-start
    err_schur += offdiagonal_frobenius(T)
print(rt_schur / repeats, err_schur / repeats)

2.3382792472839355 3.7858406276739907e-13


In [213]:
n = 10
A = np.random.randn(n,n) #+ 1j*np.random.randn(n,n)
U,_ = np.linalg.qr(A)
start = time()
H = (U+U.conj().T) / 2; S = (U-U.conj().T) / 2
mu = np.random.normal(0,1,2)
#mu[0] = np.abs(mu[0])
#mu[1] = -np.abs(mu[1])
A_mu = mu[0] * H + mu[1] * 1j*S #+ mu[2]*1j * H + mu[3]* 1j * 1j * S
D1 = eigvalsh(A_mu)
D2 = eigvalsh(H)
D2 = np.arccos(np.clip(D2,-1,1))
D2 = np.concatenate([D2,-D2])

angle = np.angle( mu[0]-1j*mu[1])
radius = np.absolute(mu[0]-1j*mu[1])
D1 = D1 / radius
D1 = np.arccos(D1)
D1_plus = angle+D1; D1_plus = D1_plus + (D1_plus > np.pi) * (- 2*np.pi) 
D1_minus = angle-D1; D1_minus = D1_minus  + (D1_minus < -np.pi) * (2*np.pi)

condition = np.array([ True if np.min(np.abs(D1_plus[x] - D2)) < np.min(np.abs(D1_minus[x] - D2)) \
                      else False for x in range(D1.size)])
D1 = np.where(condition,D1_plus,D1_minus)

D0 = eigvals(U)
print('D2: ', D2)
print('D1 plus: ', D1_plus )
print('D1 minus: ', D1_minus )
print('true angle:', np.sort(np.angle(D0)))
print('Result: ', np.sort(D1))
print(np.linalg.norm(np.sort(D1)- np.sort(np.angle(D0))))

D2:  [ 3.14159262  2.29246813  2.29246813  2.13070154  2.13070154  1.54961961
  1.54961961  1.12084268  1.12084268  0.         -3.14159262 -2.29246813
 -2.29246813 -2.13070154 -2.13070154 -1.54961961 -1.54961961 -1.12084268
 -1.12084268 -0.        ]
D1 plus:  [-1.12084268 -1.35171081 -1.54961961 -2.13070154 -2.29246813 -2.47255349
 -2.90133041 -3.14159265  2.80077296  2.63900637]
D1 minus:  [-0.23086813  0.          0.1979088   0.77899073  0.94075732  1.12084268
  1.54961961  1.78988184  2.13070154  2.29246813]
true angle: [-2.29246813 -2.13070154 -1.54961961 -1.12084268  0.          1.12084268
  1.54961961  2.13070154  2.29246813  3.14159265]
Result:  [-3.14159265 -2.29246813 -2.13070154 -1.54961961 -1.12084268  0.
  1.12084268  1.54961961  2.13070154  2.29246813]
2.247209411142166


In [188]:
print(eigenvalue_unitary_angle(U).size)

100
